In [ ]:
!pip install transformers
#!pip install torch
#!pip install pytorch_pretrained_bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
#import torch
#import torch.nn as nn
#import torch.optim as optim
#from torch.utils import data
#rom pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForQuestionAnswering, BertForPreTraining ,BertAdam
from transformers import (
    pipeline,
    AutoModelForCausalLM,
    AutoConfig,
    Trainer,
    AutoTokenizer,
    TextDataset,
    DataCollatorForLanguageModeling,
    TrainingArguments
)
import pandas as pd

In [ ]:
model_name = 'bert-base-cased'

name = f'{model_name}-title-comment'
#if IN_COLAB:
  #  !pip install transformers
 #   model_filename = f'/content/drive/MyDrive/Google colabs/{name}.pt'
#else:
model_filename = f'{name}.pt' # for when saving model to file

In [ ]:
import pickle
f = open("article_texts.txt",'rb')
texts = pickle.load(f, encoding="UTF-8")

In [ ]:
titles = []
comments = []
for text in texts:
    for i in range(len(text[2])):
        titles.append(text[0])
        comments.append(text[2][i][0])
        
zipped = list(zip(titles, comments))
        
df = pd.DataFrame(zipped, columns=['Title', 'Comment'])

In [ ]:
test_set = df.sample(n = 500)
df = df.loc[~df.index.isin(test_set.index)]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
separator = '[SEP]'

In [ ]:
class CommentsDataset(Dataset):
    
    def __init__(self, tokenizer, n=-1, max_length=1024):

        self.tokenizer = tokenizer
        self.comments = []

        for index, row in df.iterrows():
          self.comments.append(
                self.tokenizer.encode(f"[CLS]{row['Title']}{separator}{row['Comment']}{separator}")
            )
                
        if n != -1:
            self.comments = self.comments[:n]
        self.comments_count = len(self.comments)
        
    def __len__(self):
        return self.comments_count

    def __getitem__(self, item):
        return self.comments[item]

In [ ]:
dataset = CommentsDataset(tokenizer)

In [ ]:
def modelTrainer (text_path, epochs=10, model_name='gpt2', batch_size=8, cache_dir='cache'):
    model = AutoModelForCausalLM.from_pretrained (model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    data_collator = DataCollatorForLanguageModeling (tokenizer=tokenizer, mlm=False) 
    train_dataset = CommentsDataset(
        tokenizer=tokenizer,
        n = 5000,
    )
    training_args = TrainingArguments(
        output_dir=text_path,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        warmup_steps=500,
        #save_steps=2000
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset
    )
        
    trainer.train()
    trainer.save_model()

In [ ]:
modelTrainer(model_filename, epochs=20, model_name=model_name)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning


Step,Training Loss
500,3.065100
1000,0.092000
1500,0.055000
2000,0.047700
2500,0.044800
3000,0.043200
3500,0.042100
4000,0.042200
4500,0.041700
5000,0.040600


In [ ]:
model = AutoModelForCausalLM.from_pretrained("bert-base-cased-title-comment.pt")

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [ ]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [ ]:
def gen(title):
    return generator(f"{title}[SEP]", max_length=50)

In [ ]:
text = gen("[CLS]Putin to fight Trump WWE style[SEP]")[0]['generated_text']

In [ ]:
def clean(text):
    return text.split('[SEP]')[0]

In [ ]:
text

'<s>[CLS]Putin to fight Trump WWE style[SEP] “ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’'

In [ ]:
clean(text)

' ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’ ’'